# Video Processing for Eye Tracking Analysis

This notebook processes eye tracking data from a research study that combines video recordings with gaze tracking information. The main functions include:
- Converting WMV video files to MP4 format
- Processing eye tracking data from CSV files
- Overlaying gaze positions on video recordings
- Chopping videos based on annotation markers

## Prerequisites

- Required libraries: pandas, cv2 (OpenCV), numpy
- Custom utilities from ivr_utils package
- Access to the data directory containing participant recordings

## Data Organization

The code expects:

- A server data directory containing:
  - Eyetracking data in CSV format
  - Video recordings in WMV format
- CSV files contain columns:
  - Timestamp: Time in milliseconds
  - Gaze X/Y: Screen coordinates of gaze position
  - SlideEvent: Events like "StartMedia"
  - Respondent Annotations active: Used for video chopping

## Potential Improvements
Consider adding:
- Error handling for missing or corrupted files
- Progress indicators for long processing operations
- Validation of input data format and content
- Configuration options for visualization parameters
- Memory optimization for large video files
- Batch processing capabilities for multiple participants

In [1]:
import pandas as pd
import cv2
from pathlib import Path
from typing import Tuple
from ivr_utils.ivr_utils import (
    find_participant_files,
    convert_wmv_to_mp4,
    pyav_timestamps,
)
import numpy as np

# Constants
PARTICIPANT_ID = "P48"

local_data_dir = Path.cwd().parent / "data"
server_data_dir = Path(
    "/Volumes/ritd-ag-project-rd00pj-jkang71/UCL_SSID/SSID IVR Study 1"
)
output_dir = server_data_dir / "output"

assert server_data_dir.is_dir(), "Server data directory not found"

## Find Files for participant

In [2]:
eyetracking_dir = server_data_dir / "Eyetracking"

part_csv_path, part_wmv_path = find_participant_files(PARTICIPANT_ID, eyetracking_dir)
print(f"Participant CSV: {part_csv_path}")
print(f"Participant WMV: {part_wmv_path}")

Participant CSV: /Volumes/ritd-ag-project-rd00pj-jkang71/UCL_SSID/SSID IVR Study 1/Eyetracking/SSID AV1/001_P48.csv
Participant WMV: /Volumes/ritd-ag-project-rd00pj-jkang71/UCL_SSID/SSID IVR Study 1/Eyetracking/SSID AV1/Gaze Replays/Scene_P48_ScreenRecording-1_(0,OTHER,1005).wmv


# Import csv file for participant

In [3]:
points = pd.read_csv(part_csv_path, skiprows=lambda x: x < 26)

# find the "StartMedia" timestamp
row = points[points["SlideEvent"] == "StartMedia"]
timestamp_diff = row["Timestamp"].values[0]

# clean the NaN in columns
points = points.dropna(subset=["Gaze X", "Gaze Y"])

# - timestamp_diff to make the timestamp start from 0
points["Timestamp"] = points["Timestamp"] - timestamp_diff

/var/folders/6t/7h8wn9n92w5f24ml_bkwck9m0000gn/T/ipykernel_13867/1569397620.py:1: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  points = pd.read_csv(part_csv_path, skiprows=lambda x: x < 26)


## Convert to MP4 if not already found

### Dealing with the fps issues

The notebook includes analysis of frame rates across different participants:

- Timestamps are extracted from video files
- Average time differences between frames are calculated
- Equivalent FPS is computed
- Multiple participant examples (P42, P48, P6) show variation in frame rates

**P42**:

```text
Number of timestamps from AV (i.e. n frames): 24007
[0, 170, 496, 707, 807, 908, 1012, 1118, 1225, 1318]
[170 326 211 100 101 104 106 107  93 106]
Average time difference: 107.01857868866117
Standard deviation of time difference: 37.66216773314807
Equivalent FPS 9.34417193961437
```

**P48**:

```text
Number of timestamps from AV (i.e. n frames): 19417
[0, 178, 482, 583, 677, 763, 848, 937, 1022, 1116]
[178 304 101  94  86  85  89  85  94  75]
Average time difference: 114.73217964565308
Standard deviation of time difference: 45.292123590311085
Equivalent FPS 8.715950512650158
```

**p6**

```text
Number of timestamps from AV (i.e. n frames): 7497
[0, 185, 496, 590, 692, 802, 882, 977, 1071, 1168]
[185 311  94 102 110  80  95  94  97  84]
Average time difference: 122.87433297758805
Standard deviation of time difference: 46.83135463165881
Equivalent FPS 8.138396162707124
```

In [6]:
av_timestamps = pyav_timestamps(part_wmv_path)

print(f"Number of timestamps from AV (i.e. n frames): {len(av_timestamps)}")
print(av_timestamps[:10])

tdiff = np.diff(av_timestamps)
print(tdiff[:10])
print(f"Average time difference: {np.mean(tdiff)}")
print(f"Standard deviation of time difference: {np.std(tdiff)}")
print(f"Equivalent FPS {1 / (np.mean(tdiff) / 1000)}")

Number of timestamps from AV (i.e. n frames): 19417
[0, 178, 482, 583, 677, 763, 848, 937, 1022, 1116]
[178 304 101  94  86  85  89  85  94  75]
Average time difference: 114.73217964565308
Standard deviation of time difference: 45.292123590311085
Equivalent FPS 8.715950512650158


## Convert WMV to MP4

In [12]:
import logging

# Configure logging with timestamp, level and message
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

result = convert_wmv_to_mp4(part_wmv_path, "output_test3.mp4")
result.print_status()

2024-12-19 01:51:37 - INFO - Checking existing MP4 file


MP4 file already exists and is valid


## Overlaying

In [74]:
# TODO: Extract into utils function

N_FRAMES_PROC = 30 * 60  # 30 fps * 60 seconds
part_mp4_path = result.output_path

# read the video
# input_video_path = "input.mp4"
output_overlay_path = local_data_dir / f"{PARTICIPANT_ID}_output_overlay_test.mp4"

video = cv2.VideoCapture(part_mp4_path.as_posix())
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter mp4-mp4v format
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_overlay = cv2.VideoWriter(output_overlay_path, fourcc, fps, (width, height))

# # clean the NaN in columns
# points = points.dropna(subset=["Gaze X", "Gaze Y"])

# # - timestamp_diff to make the timestamp start from 0
# points["Timestamp"] = points["Timestamp"] - timestamp_diff

# loop video frames
current_point_index = 0
for frame_index in range(total_frames):
    if frame_index > N_FRAMES_PROC:
        break

    ret, frame = video.read()
    if not ret:
        break

    # time of the frame in ms
    current_time = frame_index / fps * 1000

    # check if the time has reached the next timestamp in the eyetracking csv
    while (
        current_point_index < len(points) - 1
        and points["Timestamp"].iloc[current_point_index + 1] <= current_time
    ):
        current_point_index += 1  # index=index+1

    # get the coordinates corresponding to the current timestamp
    if current_point_index < len(points):
        row = points.iloc[current_point_index]
        x, y = int(row["Gaze X"]), int(row["Gaze Y"])
        # draw a yellow circle
        cv2.circle(frame, (x, y), 50, (0, 250, 250), -1)

    out_overlay.write(frame)

video.release()
out_overlay.release()
cv2.destroyAllWindows()

## Chopping

In [78]:
N_FRAMES_PROC = 30 * 240  # 30 fps * 240 seconds

# read the video
input_path = part_mp4_path
output_video_path = local_data_dir / f"{PARTICIPANT_ID}_output_chopped_test.mp4"

video = cv2.VideoCapture(input_path)
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# # - timestamp difference
# points["Timestamp"] = points["Timestamp"] - timestamp_diff

# process frames
current_point_index = 0
skip_frames = False

for frame_index in range(total_frames):
    if frame_index > N_FRAMES_PROC:
        break

    ret, frame = video.read()
    if not ret:
        break

    current_time = frame_index / fps * 1000

    # update current_point_index based on time
    while (
        current_point_index < len(points) - 1
        and points["Timestamp"].iloc[current_point_index + 1] <= current_time
    ):
        current_point_index += 1

        # skip frames
        if (
            pd.isna(points["Respondent Annotations active"].iloc[current_point_index])
            or points["Respondent Annotations active"].iloc[current_point_index] == ""
        ):
            skip_frames = True
        else:
            skip_frames = False

    # write frame if not skipping
    if not skip_frames:
        out.write(frame)

# release resources
video.release()
out.release()
cv2.destroyAllWindows()